In [32]:
import sys
import scipy
import numpy as np
import dask.array as da
import dask.dataframe as dd
import pandas as pd
sys.path.insert(0, "../")
from GWASSimulator import GWASSimulator
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
gs = GWASSimulator("../data/genotype_data/1000G.EUR.QC.22",
                   ld_block_files="../data/ldetect-data/EUR/fourier_ls-chr22.bed",
                   keep_snps="../data/w_snplist_no_MHC.snplist.bz2")
gs.simulate()

Mapping files:  33%|███▎      | 1/3 [00:00<00:00,  5.23it/s]

> Reading genotype files...


Mapping files: 100%|██████████| 3/3 [00:00<00:00,  6.47it/s]


> Computing LD matrices...


In [ ]:
gs.update_ld_regularization(10.)
gs.simulate(n=10000)